# Workshop 3: Explore NT scenario results, modify assumptions & benchmarking

:::{note} At the end of this notebook, you will be able to:

- Navigate and analyze NT scenario results
- Interpret current discrepancies between model outputs and the TYNDP 2024 NT scenario
- Compare 2030 and 2040 scenarios using PyPSA-Explorer
- Modify model assumptions and generate new scenario results
- Apply the benchmarking framework to identify areas for model improvement

:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa pypsa-explorer pandas matplotlib numpy
```
:::

In [ ]:
# uncomment for running this notebook on Colab
# !pip install pypsa pypsa-explorer pandas matplotlib numpy

In [ ]:
import os
from datetime import datetime
import pandas as pd
import pypsa
import zipfile
from urllib.request import urlretrieve
from pdf2image import convert_from_path
from pdf2image.exceptions import PDFPageCountError
from IPython.display import Code, display
import matplotlib.pyplot as plt
from pypsa_explorer import create_app
from pathlib import Path

pypsa.options.params.statistics.round = 3
pypsa.options.params.statistics.drop_zero = True
pypsa.options.params.statistics.nice_names = False
plt.rcParams["figure.figsize"] = [14, 7]

In [ ]:
def unzip_with_timestamps(zip_path, extract_to):
    """Unzip a file while preserving original file timestamps."""
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        for member in zip_ref.infolist():
            # Extract the file
            zip_ref.extract(member, extract_to)

            # Get the extracted file path
            extracted_path = os.path.join(extract_to, member.filename)

            # Get the modification time from the zip file
            date_time = datetime(*member.date_time)
            timestamp = date_time.timestamp()

            # Set both access and modification times
            os.utime(extracted_path, (timestamp, timestamp))

In [ ]:
urls = {
    "data/results.zip": "https://storage.googleapis.com/open-tyndp-data-store/workshop-03/results.zip",
}

os.makedirs("data", exist_ok=True)
os.makedirs("scripts", exist_ok=True)
for name, url in urls.items():
    if os.path.exists(name):
        print(f"File {name} already exists. Skipping download.")
    else:
        print(f"Retrieving {name} from GCP storage.")
        urlretrieve(url, name)
        print(f"File available in {name}.")

to_dir = "data/results"
if not os.path.exists(to_dir):
    print(f"Unzipping data/results.zip.")
    unzip_with_timestamps("data/results.zip", "data/results")
print(f"NT results available in '{to_dir}'.")

to_dir = "data/open-tyndp"
if not os.path.exists(to_dir):
    print(f"Unzipping data/open-tyndp.zip.")
    unzip_with_timestamps("data/open-tyndp.zip", "data/open-tyndp")
print(f"Open-TYNDP available in '{to_dir}'.")

print("Done")

# Interactive exploration with PyPSA-Explorer

PyPSA-Explorer is an interactive web dashboard that allows you to visualize and analyze energy system networks. It provides:
- Energy balance analysis with timeseries and aggregated views
- Capacity planning visualizations by carrier and region
- Economic analysis showing CAPEX/OPEX breakdowns
- Interactive geographical network maps
- Multi-network visualisation support

Let's load the NT scenario results and explore them using PyPSA-Explorer.

In [ ]:
# Load NT scenario networks for comparison
base_path = "data/results/results/networks/"

# Load networks directly into dictionary for PyPSA-Explorer
networks = {
    "NT 2030": pypsa.Network(base_path + "base_s_all___2030.nc"),
    "NT 2040": pypsa.Network(base_path + "base_s_all___2040.nc"),
}

The PyPSA-Explorer package can be started in two modes depending on your environment:

- **Local Jupyter**: Terminal command (**recommended**) or inline display
- **Google Colab**: The dashboard will launch inline, embedded directly in the notebook

Follow the instructions below based on your environment.

In [ ]:
# Detect if running on Google Colab
try:
    from google.colab import output

    IN_COLAB = True
    print(f"This notebook is running on Google Colab !")
except ImportError:
    IN_COLAB = False
    print(f"This notebook is running locally !")

port = 8050

## For Local Users

If running locally, the **recommended approach** is to launch PyPSA-Explorer from the terminal:

```bash
pypsa-explorer data/results/results/networks/base_s_all___2030.nc:NT_2030 data/results/results/networks/base_s_all___2040.nc:NT_2040
```

This will open the dashboard in your default browser at http://localhost:8050.

**Alternative**: The cell below can also launch the dashboard inline, but the terminal method is preferred for better performance.

In [ ]:
# Terminal method recommended
USE_TERMINAL = True  # Change to False if you want to launch inline display

if not IN_COLAB and not USE_TERMINAL:
    # Local Jupyter: Inline display
    app = create_app(networks)
    app.run(jupyter_mode="tab", port=port, debug=False)

## For Google Colab Users

If running on Google Colab, you need to start PyPSA-Explorer from the terminal before the iframe can display it.

Run the following command in the Google Colab terminal:

```python
pypsa-explorer data/results/results/networks/base_s_all___2030.nc:NT_2030 data/results/results/networks/base_s_all___2040.nc:NT_2040
```

In [ ]:
if IN_COLAB:
    # Google Colab: Use built-in Dash Jupyter support to display an iframe
    output.serve_kernel_port_as_iframe(port, height=1500)

**Tip for Colab users:** To view the dashboard in fullscreen mode, click the three dots (⋮) in the top-right corner of the output cell and select **"View output fullscreen"**.

## Using the Dashboard

Once the dashboard opens, you can:

1. **Energy Balance Tab**: View production, consumption, and storage patterns
   - Switch between timeseries and aggregated views
   - Filter by energy carrier (electricity, hydrogen, etc.)
   - Filter by country/region

2. **Capacity Tab**: Analyze installed capacities
   - Compare capacity buildout between 2030 and 2040 scenarios
   - View by technology type and region
   - Identify discrepancies with TYNDP 2024 targets

3. **Economics Tab**: Examine costs and revenues
   - CAPEX and OPEX breakdowns by technology
   - Regional cost comparisons
   - Investment requirements

4. **Network Map**: Visualize the geographical network
   - Interactive map with buses, lines, and generators
   - Zoom and pan to explore specific regions
   - View transmission constraints

**Tip:** Use the scenario selector dropdown to switch between NT 2030 and NT 2040 for direct comparison.

# Benchmark results

In Workshop 2, we introduced a benchmarking framework to systematically compare Open-TYNDP model outputs against TYNDP 2024. This framework helps us:

- **Identify discrepancies**: Compare demands, installed capacities and generation volumes between model results and TYNDP targets
- **Quantify differences**: Calculate deviations by technology, country, and investment year
- **Guide improvements**: Prioritize areas where the model requires improvements

Let's apply this framework to our NT 2030 and 2040 scenario results to understand where the current model implementation aligns with or diverges from TYNDP expectations.

To do so, let's first define a function to display benchmarks.

In [ ]:
def show_benchmarks(
    fn: str,
    years: list = [2030, 2040],
    bench_path: str = "data/results/results/validation/graphics_s_all___all_years",
):
    try:
        images = [
            convert_from_path(Path(bench_path, f"{fn}_{y}.pdf"))[0] for y in years
        ]
    except PDFPageCountError:
        print("File not found, skipping...")
        return

    fig, axes = plt.subplots(1, 2)
    for ax, img in zip(axes, images):
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

## Overview

We start with an overview of the sMAPE across all carriers. This provides a high-level understanding of the magnitude of the error across all benchmarks, carriers and planning horizons. It can clearly be observed that some features were significantly improved since our introduction of the framework. However, there is also still room for improvement.

In [ ]:
display(
    convert_from_path(
        Path("data/results/results/validation/kpis_eu27_s_all___all_years.pdf")
    )[0]
)

## Final Energy Demand

Let's now examine each benchmark category in detail, starting with final energy demand across all carriers.

In [ ]:
show_benchmarks("benchmark_final_energy_demand_eu27_cy2009")

## Electricity Demand

Electricity demand is an exogenous input to the model, directly taken from TYNDP 2024 data. Therefore, we get a perfect match between the model results and TYNDP 2024.

In [ ]:
show_benchmarks("benchmark_elec_demand_eu27_cy2009")

## Hydrogen Demand

Hydrogen demand is mainly defined exogenously. Therefore, we expect a good match between the model results and TYNDP 2024. The electricity generation, on the other hand, is endogenous.

In [ ]:
show_benchmarks("benchmark_hydrogen_demand_eu27_cy2009")

## Methane Demand

Methane demand includes exogenous consumption, as well as endogenous demand for power generation and SMR (steam methane reforming).

In [ ]:
show_benchmarks("benchmark_methane_demand_eu27_cy2009")

## Power Generation Capacities

Installed generation capacities are now converging towards the values of TYNDP 2024. Both renewable and conventional capacities are now fully integrated from the PEMMDB.

In [ ]:
show_benchmarks("benchmark_power_capacity_eu27_cy2009")

## Electricity Generation

Actual electricity generation differs slightly from the TYNDP 2024 generation mix. However, the total generation values are close.

In [ ]:
show_benchmarks("benchmark_power_generation_eu27_cy2009")

## Hydrogen Supply

TODO

In [ ]:
show_benchmarks("benchmark_hydrogen_supply_eu27_cy2009")

## Methane Supply

TODO

In [ ]:
show_benchmarks("benchmark_methane_supply_eu27_cy2009")

## Biomass Supply

TODO

In [ ]:
show_benchmarks("benchmark_biomass_supply_eu27_cy2009")

## Energy Imports

Energy imports include hydrogen, methane, liquids and solids. Currently, the model cannot import biomass.

In [ ]:
show_benchmarks("benchmark_energy_imports_eu27_cy2009")

# Modify assumptions

An important exercise to working with any energy system model is changing input assumptions and this is particularly the case for any TYNDP model as assumptions are changing and many different configuration need to be tested and rerun many times.

The Open-TYNDP model has multiple ways to modify assumptions and run different scenarios. There are three main options how one can make changes to the assumptions and inputs of the model:
- **Input data:** Changing the input data in the data folder
- **Custom assumptions:** Modifying cost and technology assumptions via the `custom_cost.csv`  file
- **Custom constraints:** Adding custom constraints in the `solve_network.py` script
- **Manual adjustments:** Making selected changes to specific components via the scenario configuration file `scenario.config.yaml`

## Modifying the input data

The first way you can make changes to the model is by modifying the input data directly.

To do this, you can navigate into the `data` directory of the open-tyndp repository and find the `tyndp_2024_bundle`. 

In [ ]:
from scripts._helpers import display_tree

target_directory = "data/open-tyndp/data/tyndp_2024_bundle"
print("data")
display_tree(target_directory, max_depth=1)

Here you can freely navigate into any of the input files and exchange any of the input files with you own input file of the **same format**.

## Custom assumptions

Now, if you want to include some custom assumptions related to e.g. the capital or marginal cost or the technical parameters of the different technologies, you can specify these in a long format csv table called `custom_cost.csv` in the `data` folder.

Navigate into that folder again and look for that file. As you can see, there are already some example custom assumptions in that file:

In [ ]:
custom_cost = pd.read_csv("data/open-tyndp/data/custom_costs.csv")
custom_cost

Now, if you wanted to add you own custom assumptions, you can simply add a new row to this csv file which at least specifies the planning_horizon, the technology, the parameter, the value and the unit. 

This will then be included in the model when you rerun the Open-TYNDP workflow.

## Manual adjustments

The third way, you can modify Open-TYNDP assumptions is by including specific changes to components or specific units in the system.

These are called `adjustments` and can be added directly via th model configuration file for a your modelled scenarios.

An example of this configuration section can be found in the default configuration file:

In [ ]:
def display_code_lines(filename, language, start, end):
    with open(filename) as f:
        lines = f.readlines()
    return Code("".join(lines[start - 1 : end]), language=language)

In [ ]:
display_code_lines("data/open-tyndp/config/config.default.yaml", "yaml", 1069, 1076)

So, if you want to specify a manual adjustment to a scenario that you are modelling, you can do so by adding such a section in your `scenarios.tyndp.yaml`.

Let's have a look at that file:

In [ ]:
display_code_lines("data/open-tyndp/config/scenarios.tyndp.yaml", "yaml", 1, 100)

## Task 1: Add manual adjustments

Modify the existing `NT` scenario in the scenario configuration file `scenario.tyndp.yaml` by adding the following manual adjustments:
- TODO

Rerun the model and investigate the results using the PyPSA-Explorer.

Hint: See our last workshop notebook on how to run a Snakemake workflow.

Hint II: Remember to always start with a dry-run, i.e. adding a `-n` flag to the end of your Snakemake call.

## Custom constraints

Lastly, you might want to add some custom constraints to the model. Since we have not yet covered this aspect of PyPSA, we will not go in to much detail here and will teach you about PyPSA constraints comprehensively in one of our next workshops.

However, we still want to give you an idea of where you could add such a custom constraint that is not yet included in the model via any of the Component parameters. 

Note: An example, of a standard PyPSA constraint would be the `p_nom_max` attribute of a Link component, that will be translated by PyPSA automatically into a binding upper limit constraint for the capacity of an expandable Link component.

To do so, you can simply add your own custom code into the model. This level of customization is the great benefit of working with an open-source model.

Please navigate into the scripts directory of the open-tyndp repository and find the script called `solve_network.py`.

In [ ]:
display_code_lines("data/open-tyndp/scripts/solve_network.py", "Python", 1268, 1361)

In this script you can find a function called `extra_functionality` where different custom constraints are added to model. 

For example, we have added a custom constraint for the TYNDP offshore hubs modelling by calling a sub-function: `add_offshore_hubs_constraint`.

In the same way, you could add here your custom constraint to the model. Feel free to check out the custom constraints that are already included here.

More on this will follow in on of the next workshops.

# Solutions

## Task 1: # TODO